<a href="https://colab.research.google.com/github/Molten-Ice/Deep-Learning/blob/dev/UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [25]:
import torch
import torch.nn as nn
import numpy as np
import PIL
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
from pathlib import Path
import os
import time
import torchvision
import copy

from torch.utils.data import DataLoader
from torchgeo.datasets import NAIP, ChesapeakeDE, stack_samples
from torchgeo.datasets.utils import download_url
from torchgeo.samplers import RandomGeoSampler

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"on device: {device}")

on device: cpu


In [1]:
%pip install torchgeo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.0/612.0 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━

## Download data and creating Dataloaders

[getting_started](ttps://github.com/microsoft/torchgeo/blob/main/docs/tutorials/getting_started.ipynb)

In [4]:
data_root = tempfile.gettempdir()
naip_root = os.path.join(data_root, "naip")
naip_url = (
    "https://naipeuwest.blob.core.windows.net/naip/v002/de/2018/de_060cm_2018/38075/"
)
tiles = [
    "m_3807511_ne_18_060_20181104.tif",
    "m_3807511_se_18_060_20181104.tif",
    "m_3807512_nw_18_060_20180815.tif",
    "m_3807512_sw_18_060_20180815.tif",
]
for tile in tiles:
    download_url(naip_url + tile, naip_root)

naip = NAIP(naip_root)

  0%|          | 0/513332284 [00:00<?, ?it/s]

  0%|          | 0/521985441 [00:00<?, ?it/s]

  0%|          | 0/489865657 [00:00<?, ?it/s]

  0%|          | 0/484476647 [00:00<?, ?it/s]

In [5]:
chesapeake_root = os.path.join(data_root, "chesapeake")

chesapeake = ChesapeakeDE(chesapeake_root, crs=naip.crs, res=naip.res, download=True)

  0%|          | 0/287350495 [00:00<?, ?it/s]

In [6]:
dataset = naip & chesapeake

In [7]:
sampler = RandomGeoSampler(dataset, size=1000, length=10)

In [8]:
dataloader = DataLoader(dataset, sampler=sampler, collate_fn=stack_samples)

In [24]:
import matplotlib.pyplot as plt
sample = next(iter(dataloader))
image, target = sample["image"], sample["mask"]
print(image.shape, target.shape)
plt.imshow(image[0][2])

torch.Size([1, 4, 1000, 1000]) torch.Size([1, 1, 1000, 1000])


IndexError: ignored

In [ ]:
plt.imshow(target[0][0])

In [17]:
image

tensor([[[[140., 126., 114.,  ...,  93.,  99., 109.],
          [139., 115., 120.,  ...,  94.,  78.,  95.],
          [127., 123., 121.,  ...,  88.,  88., 100.],
          ...,
          [128., 149., 136.,  ...,  75.,  81.,  82.],
          [125., 126., 110.,  ...,  82.,  92., 105.],
          [106., 117., 119.,  ...,  59.,  72.,  88.]],

         [[166., 149., 149.,  ..., 130., 138., 149.],
          [156., 149., 148.,  ..., 117., 107., 140.],
          [153., 146., 151.,  ..., 113., 120., 143.],
          ...,
          [152., 151., 156.,  ...,  99., 103., 104.],
          [144., 159., 126.,  ..., 102., 121., 126.],
          [132., 132., 144.,  ...,  75.,  92., 118.]],

         [[108., 103., 101.,  ...,  82.,  90., 101.],
          [113., 103.,  95.,  ...,  83.,  73.,  84.],
          [104., 100.,  97.,  ...,  87.,  80.,  88.],
          ...,
          [109., 106., 115.,  ...,  79.,  87.,  85.],
          [103., 119.,  96.,  ...,  78.,  81.,  96.],
          [ 94., 105., 100.,  ...

In [10]:
for sample in dataloader:
    image = sample["image"]
    target = sample["mask"]
    print(image.shape, target.shape)
    break

torch.Size([1, 4, 1000, 1000]) torch.Size([1, 1, 1000, 1000])
